In [19]:
import requests
import scrapy
import pandas as pd

from bs4 import BeautifulSoup
from csv import writer


# https://developers.whatismybrowser.com/useragents/parse/2899915-chrome-windows-blink

def init_scraper():
    UserAgent =  ({'User-Agent':
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
            'Accept-Language': 'en-US, en;q=0.5'})
    url = "https://www.dia.es/compra-online/frescos/carne/cf?page=1"
    soup = get_soup(url, UserAgent)
    return soup

def get_soup(url, agent):
    web = requests.get(url, headers = agent)
    soup = BeautifulSoup(web.content, 'html.parser')
    return soup

def get_products(soup):
    listOf_products = []
    data = soup.find(class_ = "product-list__item").findAll()
    #print(data)
    count = 0
    for tag in range(len(data)):
        listOf_products[tag] = data[tag]
        print("print", count)
        count = count + 1
    
    return listOf_products
        
soup = init_scraper()
#list_products = get_products(soup)

print(soup.prettify())
print(soup.title)
#print(soup)


<!DOCTYPE html>
<html class="classic" lang="es">
 <head>
  <script src="https://cdn.cookielaw.org/consent/fc06e8ab-0d54-4f22-8dc9-b371a5837b16/OtAutoBlock.js" type="text/javascript">
  </script>
  <script charset="UTF-8" data-domain-script="fc06e8ab-0d54-4f22-8dc9-b371a5837b16" src="https://cdn.cookielaw.org/scripttemplates/otSDKStub.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   function OptanonWrapper() {window.dataLayer.push({event:'OneTrustGroupsUpdated'});}
  </script>
  <!-- Necesario dado que hay slots que solo cargan cuando se usa un dispositivo mobile, y no son editables desde SE
         a menos que se cargue la pagina mobile. Cuando los componentes sean responsive, todo el codigo marcado *SEMR*
         se puede borrar -->
  <!-- SEMR -->
  <title>
   Carne - Productos frescos - Supermercados Dia | Página 2
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Compra online productos de carnes y aves 

In [22]:
print(soup.title.string)


		Carne - Productos frescos - Supermercados Dia | Página 2


In [23]:
for link in soup.find_all('a'):
    print(link.get('href'))

#skip-to-content
#skiptonavigation
https://www.google.es/chrome
https://www.mozilla.org/firefox/
/compra-online/my-account/orders
/compra-online/
https://www.dia.es/tiendas
/compra-online/login/singlelogin
/compra-online/login/singlelogin/registerview
/compra-online/my-account/orders
/compra-online/my-account/saved-lists
https://www.dia.es/clubdia
/compra-online/cart
#
/compra-online/my-account/orders
#
/compra-online/cart
/compra-online/login/singlelogin
/compra-online/login/singlelogin/registerview
/compra-online/my-account/orders
/compra-online/my-account/saved-lists
https://www.dia.es/clubdia
None
None
/compra-online/frescos/cf
/compra-online/frescos/carne/cf?show=All
/compra-online/frescos/carne/cf?page=0&disp=
/compra-online/frescos/carne/cf?page=2&disp=
/compra-online/frescos/carne/pollo/cf
/compra-online/frescos/carne/pavo/cf
/compra-online/frescos/carne/cerdo/cf
/compra-online/frescos/carne/vacuno/cf
/compra-online/frescos/carne/mixto/cf
/compra-online/frescos/carne/pollo/cf
/

In [24]:
print(soup.get_text())










		Carne - Productos frescos - Supermercados Dia | Página 2



























































text.skipToContent
text.skipToNavigation











/dia/cart/
UA-10364527-1
Su navegador no es compatible con esta página web. Por favor, actualice su versión o instale un navegador moderno que soporte estándares (Chrome, Firefox).



Selecciona Código Postal para ver la fecha y hora más próxima de entrega

























Folletos y tiendas




Inicio Sesión





Inicio Sesión
Registrarme


Últimas compras


Listas guardadas

Club Día










0















									Catálogo Productos
























Repetir compra






0,00 €

0


















Tu carrito está vacío
Añade productos al mismo.







			Total artículos ()
			0,00 €


			Ahorro:0,00 €


			Gastos de envío:
			0,00 €




			Ver pedido













Inicio Sesión
Registrarme



Últimas compras


Listas guardadas

Club Día












Recuerda que si cambias de modalidad de e

In [28]:
for tag in soup.find_all(True):
    print(tag.name)

html
head
script
script
script
title
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
meta
link
link
link
link
link
link
link
link
link
link
link
link
script
script
script
body
sct-config
script
sct-modal
script
script
script
noscript
iframe
script
script
input
div
div
div
div
div
a
a
header
script
script
script
cas:gateway
input
input
input
script
span
span
span
span
div
p
strong
a
a
div
div
div
p
span
span
span
div
span
span
span
div
a
svg
g
path
path
div
div
a
img
div
ul
li
a
li
div
button
span
div
div
ul
li
a
a
li
a
li
a
li
a
a
div
span
span
input
input
input
input
div
div
a
span
span
span
span
span
script
script
script
script
script
script
div
meta
form
meta
input
input
div
div
a
svg
g
path
path
span
div
script
script
div
span
span
span
div
div
div
div
div
div
div
script
span
span
div
div
a
div
ul
li
li
ul
div
ul
li
span
span
li
span
li
span
div
a
div
div
button
div
div
ul
ycommerce:testid
li
a
a
li
a
li
a
li
a
a
div
div
span
span
h3
b

a
span
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
span
button
div
div
button
span
div
span
a
span
ul
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
span
button
div
div
button
span
div
span
a
span
ul
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
a
li
a
span
ul
li
div
button
span
li
a
span
li
a
li
a
li
a
li
a
li
a
li
span
button
div
div
button
span
div
span
a
span
ul
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
a
li
span
button
div
div
button
span
div
span
a
spa

In [45]:

print(soup.find("span", class_ = "details"))
print(soup.find("p", class_ = "price"))

soup1 = soup.find("div", class_="product-list__item")
link = soup1.find("a", class_ = "productMainLink")

print(link)

#print(soup1)

#print(soup1.find("span", class_ = "details"))

#soup.find('div', attrs = {'details'})

<span class="details">
					LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr</span>
<p class="price">

							2,25 €</p>
<a class="productMainLink" href="/compra-online/frescos/carne/pollo/p/209533" title="LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr">
<div class="column-left">
<div class="thumb">
<img alt="LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr" class="crispImage" itemprop="image" onerror="missingImage(this);" src="https://s0.dia.es/medias/h84/h24/10664241954846.jpg" title="LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr"/>
</div>
<div class="rating-container">
<div class="rateyo-readonly" data-rating="5.0"></div>
</div>
</div>
<div class="column-right">
<span class="details">
					LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr</span>
<div class="price_container">
<p class="price">

							2,25 €</p>
<p class="pricePerKilogram">9,00 €/Kg.</p>
</div>
<ul class="product-grid-labels

In [54]:
product
for div in soup.find_all("div", class_ = "product-list__item"):
    print("\n\nProduct number: ", product)
    details = div.find("span", class_ = "details")
    print(details.text)
    #print(div.find("p", class_ = "price"))
    #print(div.find("p", class_ = "pricePerKilogram"))
    product = product + 1
    #for product in div.find_all("div", class_ = "")



Product number:  0

					LA CARLOTEÑA redondo de pollo asado relleno de bacon envase 250 gr


Product number:  1

					SELECCIÓN DE DIA alas de pollo adobadas bandeja 450 gr


Product number:  2

					SELECCIÓN DE DIA filetes de cinta de lomo de cerdo bandeja 400 gr


Product number:  3

					SELECCIÓN DE DIA Entrecot de vacuno (peso aprox. 330 gr)


Product number:  4

					SERRANO fiambre de lomo al horno sobre 240 gr 


Product number:  5

					SELECCIÓN DE DIA filetes de pechuga de pollo bandeja (peso aprox. 825 gr)


Product number:  6

					SELECCIÓN DE DIA tira de costilla de cerdo a las finas hierbas (peso aprox. 1.4 Kg)


Product number:  7

					Filetes de pechuga de pollo de antaño bandeja 400 gr


Product number:  8

					SELECCIÓN DE DIA hamburguesas de cerdo y vacuno bandeja 6 uds 540 gr 


Product number:  9

					SELECCIÓN DE DIA hamburguesas de vacuno bandeja 6 uds 540 gr 


Product number:  10

					Lomo de vaca madurado (peso aprox. 550 gr)


Product number:  11

	

In [52]:
#https://www.dia.es/compra-online/robots.txt

SyntaxError: invalid syntax (Temp/ipykernel_20556/3767976437.py, line 1)